In [1]:
import pandas as pd

import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span, DocBin

from IPython.display import clear_output

from datetime import datetime

import PyPDF2
import tabula

import os
import re
import json

c:\users\justin\appdata\local\programs\python\python36\lib\site-packages\cupy\_environment.py:214: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  'CUDA path could not be detected.'
c:\users\justin\appdata\local\programs\python\python36\lib\site-packages\cupy\_environment.py:214: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  'CUDA path could not be detected.'


In [2]:
df = pd.read_csv('..\\data\\ocred\\files_df.csv', index_col = 0)
spacy.prefer_gpu()
nlp = spacy.load("nl_core_news_lg")

## Request metadata



Metadata about wob requests can be usefull to collect. Luckily, when the government agency to which the request was send to completes the request, it also gives a decision document and a inventory list in addition to the documents that were requested. With these two documents the reason for a request, the date on which the request was received and completed, the number of documents concidered, the number of documents (partially) released, and the number of documents not released can be extracted.

# Reason for request
The reason for the wob request can be found in the decision document. In this document it states in one sentence a summary of what has been requested. This summary is what needs to be extracted. This summary is usually indicated by a keyword or keywords. These keywords come down to dutch versions of "requested", "information about", or "publication of". What follows is a list of all keywords used in dutch:
- verzocht
- u verzoekt
- om informatie over
- uw verzoek ziet
- om openbaarmaking van
- more to come
When one of these keywords are found, all following text is extracted until a the next period occurs. To do this, a regular expression was used. 

keyword([^.]+?)\\.

Where keyword is one of the words or phrases listed above. The expression first finds one of these keywords and then matches any alpha-numerical character until the first period is found. Before the regular expression can be used however, the text needs some preprocessing. First, exessive newlines are removed. Second, all letters are converted to lowercase letters. This is done so that the regular expression will match the keywords even though in the text the the keywords are written with capitalization. This has to be done as regular expressions are case sensitive. Last, for any word or abbreviation in the text that includes a period where the period does not indicate the end of a sentence, said period have to be removed otherwise it will trip up the regular expression. After this, the neccessary information can be extracted.

After the regular expressions have been run, the matches need to be checked for duplicates. This can happen when the decision document states for example "the request if for information about [...]". In this case the same sentence would be matched for the keyword "request" and "information about". This would match the same sentence twice so only one of these is needed and the other is removed.

To evaluate the extractor, a different method than the other extractors was used: the panoptic quality metric. With this metric precision, recall, and F1 scores are still calculated, but with different method. It uses the overlap between the ground truth and the prediction to measure the performance of a model. First the intersect and union of the ground truth and prediction are calculated. The intersect is the overlap between the two and the union is the combination of the two. With this the Intersection over Union (IoU) metric is calculated. TODO[INSERT CALCULTION HERE]. The IoU can then be compared to a threshold value. If the IoU is higher then the treshold it counts as a true positive, if it is lower it counts as both a false negative and a false positive. Presicion, recall and F1 score can then be calculated. 
https://medium.com/@danielmechea/panoptic-segmentation-the-panoptic-quality-metric-d69a6c3ace30

# Relevant dates
There are two relevant dates that can be extracted from the decision document: the date on which the request was received and the date on which the request was completed. The decision document has these two dates at the beginning. The completion date is same date as when the document was made so that is always the very first date in the document. The date of request is always in the first sentence of the document as they all start with as follows: "In your letter of 01 januari 2022". This means that the first and second date that are found in the document are the relevant dates to extract. Sometimes the date a request was send is not the same as the date the request was received. This happens when the request is send by post. In this case the decision document states: "in your letter of 01 januari 2022, received on 05 januari 2022". The following regular expression was used to check if this is the case:

'ontvangen op ([^.]+?)\,'

 In this case the first and third date are the relevant dates. To actually extract the dates, the dates extractor described here TODO[REF HERE] was used. These dates can then also be used to check how long the request took to fulfull and if it was done in the time that they have. 


# Number of documents
The inventory lists contain an overview of all documents that were found that fall in the scope of the request. The list also has information about which documents were made public, which were made partially public, which were not made public and also the documents that were already public.


## Results

One problem the used approach is sensetive to is mistakes made in the OCR process. The regular expressions look some keywords and, when those are found, the end of the sentence. If in the OCR process a mistake was made in on of the keywords or if the period at the end of the sentence wasn't recognized as a period, the regular expression won't match it even though it should. 

In [3]:
# some of the dirs id wob request as 1, 2,3 and some do it as 1.0, 2.0, 3.0
# this makes all of the dirs use the first method
def fixNamingSceme():
    base = 'F:\\Data files\\Master thesis\\verzoeken\\'
    for dir in os.listdir(base):
        for folder in os.listdir(base + dir):
            if os.path.isdir(base + dir + '\\' + folder):
                os.rename(base + dir + '\\' + folder, base + dir + '\\' + folder.split('.')[0])


In [4]:
# dates matcher
def getDates(text, nlp):
    months = ['januari', 'februari', 'maart', 'april', 'mei', 'juni', 'juli', 'augustus', 'september', 'oktober', 'november', 'december',
         'january', 'february', 'march', 'april', 'may', 'june', 'juli', 'august', 'september', 'october', 'november', 'december',
         'jan', 'feb', 'mrt', 'apr', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec', 'okt']
    days = ['maandag', 'dinsdag', 'woensdag', 'donderdag', 'vrijdag', 'zaterdag', 'zondag',
       'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']

    datesPattern = [ 
           {"IS_DIGIT": True}, 
           {"LOWER" : {"IN" : months}},
           {"IS_PUNCT" : True, "OP" : "?", "TEXT":'.'},
           {"IS_DIGIT": True}]
    matcher = Matcher(nlp.vocab)
    matcher.add("Dates", [datesPattern])

    doc = nlp(text)
    matches = matcher(doc)

    return [doc[start:end].text for match_id, start, end in matches]


In [5]:
# because the extractor used periods as indicators, abbreviations like N.V.T. need to be removed
# this function converts it to N V T while keeping the periods at the end of sentences
def removeAbbreviation(text):
    sentence = ''

    # split sentence in words
    for word in text.split(' '):

        # if there is no period in the word, add it back to sentence
        if word.count('.') == 0 or '\n' in word:
            sentence += word + ' '
            continue
        
        # when there is one period in word
        elif word.count('.') == 1:

            # if it is at the end, keep it and add word to sentence
            if word[-1] == '.':
                sentence += word + ' '
            
            # if its in the middle replace it with a space
            else:
                word = word.replace('.', ' ')
                sentence += word + ' '
        
        # if there are  more than 1 periods, replace them all
        else:
            word = word.replace('.', ' ')
            sentence += word + ' '
            
    return sentence


In [6]:
def getRequestReason(text):

    # text = rawText.replace('\n', ' ')
    text = text.lower()
    text = removeAbbreviation(text)

    patterns = ['verzocht([^.]+?)\.', 'u verzoekt([^.]+?)\.', 'om informatie over([^.]+?)\.', 'uw verzoek ziet([^.]+?)\.', 'om openbaarmaking van([^.]+?)\.']
    matches = []

    # get matches for all keywords
    for pattern in patterns:
        matches += re.findall(pattern, text)

    uniqueMatches = []

    # check all matches against eachother
    for i in range(len(matches)):
        for j in range(len(matches)):
            if i == j:
                continue
            
            # if there is a duplicate we do not add it to uniqueMatches
            if matches[i] in matches[j]:
                break

        # add match to uniqueMatches if the j loop completes
        else:
            uniqueMatches.append(matches[i])

    return matches



In [7]:
# this function gets a random decision doc from a given ministry
def getRandomDecisionDoc(ministrie, rState=None):

    # set paths 
    baseDFPath = '..\\data\\openstate data\\'
    basePDFPath = 'F:\\Data files\\Master thesis\\verzoeken\\'

    # get df of ministry
    for f in os.listdir(baseDFPath):
        if ministrie in f.lower():
            file = f
            break

    # get dir of ministry 
    for d in os.listdir(basePDFPath):
        if ministrie in d.lower():
            dir = basePDFPath + d + '\\'
            break
        
    # load in dataframe of ministry and get random sample
    testDf = pd.read_excel(baseDFPath + file)
    testDf.columns = [x.replace('\n', '') for x in testDf.columns]
    # testDf = testDf[['WOB Verzoek', 'Soort aanvraag', 'URL']]
    # testDf = testDf.dropna(subset=['Soort aanvraag'])
    if rState:
        s = testDf.sample(1, random_state=rState)
    else:
        s = testDf.sample(1)

    # get request number to find correct pdf
    requestNr = s['WOB Verzoek'].values[0]

    if not os.path.exists(dir + str(requestNr)):
        return getRandomDecisionDoc(ministrie, rState)

    # find the desicion document
    for p in os.listdir(dir + str(requestNr)):

        # if found, save it in pdfPath
        if 'besluit' in p.lower() and 'bijlage' not in p.lower():
            pdfPath = p
            break
    
    # if there is no desicion document, try again
    else:
        return getRandomDecisionDoc(ministrie, rState)

    return s, dir + str(requestNr) + '\\', pdfPath


In [8]:
def textExtract(dir, name, startPage = 0, endPage = 10):
    txtName = '.'.join(name.split('.')[0:-1])
    txtName = dir + txtName + f'-pages{startPage}-{endPage}' + '.txt'
    
    # clean up from previous messups
    # for f in os.listdir(dir):
    #     if f.endswith('.txt') and f != txtName:
    #         os.remove(dir + f)


    # check if text is already extracted
    if not os.path.exists(txtName):
        os.system(f'pdftotext -f {startPage} -l {endPage} -raw "{dir}{name}" "{txtName}"')

    # open file and return content
    with open(txtName, 'r', encoding='utf8') as f:
        text = f.read()
    return text


In [9]:
def convertDate(date):
    months = {
        'januari':'01','jan':'01',
        'februari':'02','feb':'02',
        'maart':'03','mrt':'03',
        'april':'04','apr':'04',
        'mei':'05','mei':'05',
        'juni':'06','jun':'06',
        'juli':'07','jul':'07',
        'augustus':'08','aug':'08',
        'september':'09','sep':'09',
        'oktober':'10','okt':'10',
        'november':'11','nov':'11',
        'december':'12','dec':'12'
    }


    date = date.split(' ')
    date[1] = months[date[1].lower()]
    return date[2] + '-' + date[1] + '-' + date[0]


def days_between(d1, d2):
    d1 = convertDate(d1)
    d2 = convertDate(d2)
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days)


In [10]:

def dateInformation(text):

    # find dates in text with date extractor
    matches = getDates(text, nlp)

    # first found date is date when document was written
    # the date the wob request was completed
    if len(matches) < 3:
        return None, None, None, None
    completedDate = matches[0]

    # check if request was received on a different date then when it was send
    # this is the case if it states "ontvangen op"
    receivedDate = re.findall('ontvangen op ([^.]+?)\,', text)
    if not receivedDate:
        receivedDate = matches[1]
    else:
        receivedDate = matches[2]

    # calculate days between request and completion
    daysTaken = days_between(completedDate, receivedDate)

    inTime = daysTaken <= 42
    return receivedDate, completedDate, daysTaken, inTime


In [11]:
def nDocs(text):

    # look for the first mention of a number of documents
    nDocuments = re.findall('[0-9]+? document', text)

    # return the first if found, else return None
    if nDocuments:
        return int(nDocuments[0].split(' ')[0])
    else:
        return None

In [12]:
def getDocByNumber(ministry, number):
    base = 'F:\\Data files\\Master thesis\\verzoeken\\'
    for d in os.listdir(base):
        if ministry in d.lower():
            ministryDir = d
            break

    dir = 'F:\\Data files\\Master thesis\\verzoeken\\' + ministryDir + '\\' + number + '\\'
    
    for p in os.listdir(dir):

        # if found, save it in pdfPath
        if 'besluit' in p.lower() and 'bijlage' not in p.lower():
            return textExtract(dir, p, 11, 13)
    
    # if there is no desicion document, return None
    else:
        return None

In [13]:
# find number of pages in pdf documents
def getNumberOfPages(path):
    nPages = 0

    # gets list of pdf files in a directory
    pdfs = [x for x in os.listdir(path) if x.endswith('.pdf')]
    
    # open all files and count pages
    for file in pdfs:

        with open(path + file, 'rb') as f:
            pdf = PyPDF2.PdfFileReader(f, strict=False)
            nPages += pdf.numPages
    return nPages





In [18]:
def inventory(path, pdf):

    # words to look for
    rating = {'deels openbaar':0, 'niet openbaar':0, 'openbaar':0, 'reeds openbaar':0, 
            'geweigerd':0, 'gedeeltelijk openbaar':0,
            'volledig openbaar': 0}
    
    # if an inventory document exists, use that
    for file in path:
        if 'inventarislijst' in file.lower():
            rating = inventoryListToDataframe(path + file, rating)
    
    # else try to find inventory table in decision doc
    else:
        rating = inventoryListToDataframe(path + pdf, rating)

    if not rating:
        return None, None, None, None

    # combine categories
    public = (rating['openbaar'] + rating['volledig openbaar']) - (rating['deels openbaar'] + rating['niet openbaar'] + rating['reeds openbaar'])
    notPublic = rating['niet openbaar'] + rating['geweigerd'] + rating['reeds openbaar']
    partialPublic = rating['deels openbaar'] + rating['gedeeltelijk openbaar']
    total = public + notPublic + partialPublic
    
    if total == 0:
        return None, None, None, None
    
    return public, notPublic, partialPublic, public + notPublic + partialPublic

In [20]:
 def inventoryListToDataframe(pdf, rating):
    tables = tabula.read_pdf(pdf, pages='all')

    if len(tables) == 0:
        return None

    for table in tables:
        for col in table.columns:
            col = list(table[col])
            col = [str(x).lower() for x in col]
            for key in rating:
                rating[key] += col.count(key)
 
    return rating


In [24]:
def test():
    mins = ['lnv','az','buza','bzk','ezk','fin','ienw','jenv','ocw','szw','vws', 'corona']
    for min in mins:
        text = ''

        # find a document with at least 20 chars
        while len(text) < 20:
            clear_output()
            groundTruth, pdfPath, pdfName = getRandomDecisionDoc(min)
            print(f'Path: {pdfPath}{pdfName}')
            print('\n______________________________')

            # extract text from document
            text = textExtract(pdfPath, pdfName)
        
        reason = getRequestReason(text)
        receivedDate, completedDate, daysTaken, inTime = dateInformation(text)
        nDocuments = nDocs(text)
        nPages = getNumberOfPages(pdfPath)
        public, notPublic, partialPublic, total = inventory(pdfPath, pdfName)
        
        if daysTaken and nDocuments:
            daysPerDoc = round(daysTaken / nDocuments, 2)
        elif total and daysTaken:
            daysPerDoc = round(daysTaken / total, 2)
        else:
            daysPerDoc = None

        desc = groundTruth['Soort aanvraag'].values[0]
        received = groundTruth['Datum van binnenkomst'].values[0]
        awnser = groundTruth['Datum van antwoord'].values[0]
        daysTakenGT = groundTruth['Aantal dagen in behandeling'].values[0]
        inTimeGT = groundTruth['Binnen de termijn afgehandeld'].values[0]
        nDocsConsidered = groundTruth['Aantal overwogen documenten'].values[0]
        daysPerDocGT = groundTruth['Aantal dagen nodig gehad per document'].values[0]
        nPagesGT = groundTruth["Omvang document (aantal pagina's)"].values[0]
        publicGT = groundTruth["Volledig verstrekte documenten"].values[0]
        notPublicGT = groundTruth["Niet verstrekte documenten"].values[0]
        partialPublicGT = groundTruth["Deels verstrekte documenten"].values[0]

        print(f'Ground truth description: {desc}')
        print(f'Extracted description: {reason}')
        print('\n______________________________')
        print(f'Ground truth received: {received}')
        print(f'Extracted received: {receivedDate}')
        print('\n______________________________')
        print(f'Ground truth awnser: {awnser}')
        print(f'Extracted awnser: {completedDate}')
        print('\n______________________________')
        print(f'Ground truth days taken: {daysTakenGT}')
        print(f'Extracted days taken: {daysTaken}')
        print('\n______________________________')
        print(f'Ground truth in Time: {inTimeGT}')
        print(f'Extracted in time: {inTime}')
        print('\n______________________________')
        print(f'Ground truth n docs considered: {nDocsConsidered}')
        print(f'Extracted n docs considered (decision doc): {nDocuments}')
        print(f'Extracted n docs considered (inventory doc): {total}')
        print('\n______________________________')
        print(f'Ground truth days per doc: {daysPerDocGT}')
        print(f'Extracted days per doc: {daysPerDoc}')
        print('\n______________________________')
        print(f'Ground truth pages: {nPagesGT}')
        print(f'Extracted pages: {nPages}')
        print('\n______________________________')
        print(f'Ground truth public docs: {publicGT}')
        print(f'Extracted public docs: {public}')
        print('\n______________________________')
        print(f'Ground truth partial docs: {partialPublicGT}')
        print(f'Extracted partial docs: {partialPublic}')
        print('\n______________________________')
        print(f'Ground truth not public docs: {notPublicGT}')
        print(f'Extracted not public docs: {notPublic}')
        print('\n______________________________')
        print('\n\n\n')
        # print(text)

        x = input()
        if x == 'q':
            break
        else:
            clear_output()
test()

Path: F:\Data files\Master thesis\verzoeken\Wob verzoeken LNV\146\Bes-besluit+op+wob-verzoek+over+meitellingen+en+ubn-administratie+in+geesteren.pdf

______________________________


Got stderr: Jun 02, 2022 3:59:38 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JPEG2000 image: Java Advanced Imaging (JAI) Image I/O Tools are not installed
Jun 02, 2022 3:59:38 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JPEG2000 image: Java Advanced Imaging (JAI) Image I/O Tools are not installed
Jun 02, 2022 3:59:38 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JPEG2000 image: Java Advanced Imaging (JAI) Image I/O Tools are not installed



Ground truth description: Verzoek om informatie over de meitellingen en de administratie van het Uniek Bedrijfsnummer (UBN) van een locatie in Geesteren over de periode 2005 tot en met heden
Extracted description: [' over de\nmeitellingen en ubn-administratie van de locatie, walemaatweg 2, te geesteren\nover de periode 2005 tot en met heden', ' om bekendmaking van de naam\nvan de verzoeker en om een kopie het ingediende wob-verzoek']

______________________________
Ground truth received: 2020-12-29T00:00:00.000000000
Extracted received: 29 december 2020

______________________________
Ground truth awnser: 2021-03-04T00:00:00.000000000
Extracted awnser: 4 maart 2021

______________________________
Ground truth days taken: 65
Extracted days taken: 65

______________________________
Ground truth in Time: Nee
Extracted in time: False

______________________________
Ground truth n docs considered: 5
Extracted n docs considered (decision doc): None
Extracted n docs considered (inventory doc)


één document

U vraagt om  bij keywords

In [26]:
 def inventoryListToDataframe(pdf, rating):
    tables = tabula.read_pdf(pdf, pages='all')
    return tables
    if len(tables) == 0:
        return None

    for table in tables:
        for col in table.columns:
            col = list(table[col])
            col = [str(x).lower() for x in col]
            for key in rating:
                rating[key] += col.count(key)
 
    return rating


In [27]:
rating = {'deels openbaar':0, 'niet openbaar':0, 'openbaar':0, 'reeds openbaar':0, 
            'geweigerd':0, 'gedeeltelijk openbaar':0,
            'volledig openbaar': 0}
inventoryListToDataframe('F:\\Data files\\Master thesis\\verzoeken\\Wob verzoeken LNV\\146\\Bes-besluit+op+wob-verzoek+over+meitellingen+en+ubn-administratie+in+geesteren.pdf', rating)

Got stderr: Jun 02, 2022 4:02:54 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JPEG2000 image: Java Advanced Imaging (JAI) Image I/O Tools are not installed
Jun 02, 2022 4:02:54 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JPEG2000 image: Java Advanced Imaging (JAI) Image I/O Tools are not installed
Jun 02, 2022 4:02:54 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JPEG2000 image: Java Advanced Imaging (JAI) Image I/O Tools are not installed



[   Unnamed: 0  Nr.  Unnamed: 1                          Document  Unnamed: 2  \
 0         NaN  1.0         NaN  Diertelling periode 2005 - heden         NaN   
 1         NaN  2.0         NaN    UBN-administratie 2005 - heden         NaN   
 2         NaN  3.0         NaN     UBN-administratie 2015 - 2020         NaN   
 3         NaN  4.0         NaN     UBN-administratie 2015 - 2020         NaN   
 4         NaN  5.0         NaN    UBN-administratie 2005 - heden         NaN   
 
               Beoordeling  Unnamed: 3 Wob-artikel  Unnamed: 4  
 0      Volledig\ropenbaar         NaN        geen         NaN  
 1  Gedeeltelijk\ropenbaar         NaN      10.1.c         NaN  
 2  Gedeeltelijk\ropenbaar         NaN      10.1.c         NaN  
 3  Gedeeltelijk\ropenbaar         NaN      10.1.c         NaN  
 4  Gedeeltelijk\ropenbaar         NaN      10.1.c         NaN  ,
                               vleesvarkens tot 50 kg  600  stuks
 0          nog nooit gedekte fokzeugen 50 kg of meer  

In [16]:
def getDataFrameOfMinistry(ministrie):

    baseDFPath = '..\\data\\openstate data\\'

    for f in os.listdir(baseDFPath):
        if ministrie in f.lower():
            file = f
            break

    return pd.read_excel(baseDFPath + file)


t = getDataFrameOfMinistry('lnv')

t.columns = [x.replace('\n', '') for x in t.columns]
t.columns

Index(['WOB Verzoek', 'Onderwerp', 'Datum van binnenkomst',
       'Datum van antwoord', 'Aantal dagen in behandeling',
       'Binnen de termijn afgehandeld', 'Omvang document (aantal pagina's)',
       'Volledig verstrekte documenten', 'Deels verstrekte documenten',
       'Niet verstrekte documenten', 'Aantal overwogen documenten',
       'Aantal dagen nodig gehad per document', 'Soort aanvraag',
       'Bijzonderheden', 'URL'],
      dtype='object')

In [48]:
gt = getDataFrameOfMinistry('corona')
gt[gt['WOB Verzoek'] == 16]


64


,WOB Verzoek,Onderwerp,Betreffend\nministerie,Datum van binnenkomst,Datum van antwoord,Aantal dagen \nin behandeling,Binnen de \ntermijn afgehandeld,Omvang document (aantal pagina's)\n,Volledig verstrekte documenten,Deels verstrekte documenten,Niet verstrekte documenten,Aantal overwogen \ndocumenten,Aantal dagen nodig \ngehad per document,Soort aanvraag,Bijzonderheden,URL
16,16,VION-groep,LNV,2020-10-26 00:00:00,2021-10-12 00:00:00,351,Nee,216,1 kaart\n1 persbericht\n1 omschrijving,2 brieven\n45 mail(wisselingen)\n2 agenda's\n6...,Niet verstrekt want reeds openbaar:\n2 persber...,94.0,3.734043,Verzoek om informatie inzake de VION-groep,Betreft een tweede deelbesluit,https://wobcovid19.rijksoverheid.nl/publicatie...
